# Image categorisation

*  JantaHack Computer Vision by analyticsvidhya
*  Classify emergency and non-emergency vehicle from images

In [0]:
import pandas as pd
import numpy as np
import random
import math
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPool2D, BatchNormalization
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils.np_utils import to_categorical

from keras.applications.resnet50 import ResNet50
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Input, Dense, GlobalAveragePooling2D
from keras.optimizers import adam
from keras import backend as K

import seaborn as sns
from matplotlib import pyplot as plt

In [0]:
# numpy random number geneartor seed
# for reproducibility
np.random.seed(123)

# set plot rc parameters
# jtplot.style(grid=False)
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['axes.facecolor'] = '#232323'
#plt.rcParams['axes.edgecolor'] = '#FFFFFF'
plt.rcParams['figure.figsize'] = 10, 7
plt.rcParams['legend.loc'] = 'best'
plt.rcParams['legend.framealpha'] = 0.2
plt.rcParams['text.color'] = '#666666'
plt.rcParams['axes.labelcolor'] = '#666666'
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['xtick.color'] = '#666666'
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.color'] = '#666666'
plt.rcParams['ytick.labelsize'] = 14

# plt.rcParams['font.size'] = 16

sns.color_palette('dark')
%matplotlib inline

## Load data

In [54]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
data_dir = '/content/drive/My Drive/JantaHack Computer Vision/data'
img_dir = '/content/drive/My Drive/JantaHack Computer Vision/data/images'

In [56]:
train_img = pd.read_csv(data_dir+'/train.csv')
test_img = pd.read_csv(data_dir+'/test_vc2kHdQ.csv')
train_img.shape, test_img.shape

((1646, 2), (706, 1))

In [0]:
train_img['emergency_or_not'] = train_img['emergency_or_not'].apply(lambda x: str(x))

In [58]:
train_img.head()

,image_names,emergency_or_not
0,1503.jpg,0
1,1420.jpg,0
2,1764.jpg,0
3,1356.jpg,0
4,1117.jpg,0


## Load Image data

### Load image to array

In [59]:
im1 = load_img(img_dir+'/23.jpg')
im1_array = img_to_array(im1)
im1_array.shape

(224, 224, 3)

In [0]:
# x = np.zeros(shape = [1646, 224, 224, 3])

In [0]:
# for i, img in enumerate(train_img['image_names'].values):
#     x[i] = img_to_array(load_img('data/images/'+img))

### Load image using generators

In [0]:
datagen = ImageDataGenerator(rescale=1./255.,
                             validation_split=0.25,
                             rotation_range=30,
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                             shear_range=0.2,
                             zoom_range=0.2,
                             channel_shift_range=0.2,
                             horizontal_flip=True,
                             vertical_flip=True)

In [63]:
# train generator
train_generator=datagen.flow_from_dataframe(dataframe=train_img,
                                            directory=img_dir,
                                            x_col="image_names",
                                            y_col="emergency_or_not",
                                            subset="training",
                                            batch_size=32,
                                            seed=42,
                                            shuffle=True,
                                            class_mode="binary",
                                            target_size=(224,224),
                                            color_mode='rgb')
# validation data generator
valid_generator=datagen.flow_from_dataframe(dataframe=train_img,
                                            directory=img_dir,
                                            x_col="image_names",
                                            y_col="emergency_or_not",
                                            subset="validation",
                                            batch_size=32,
                                            seed=42,
                                            shuffle=True,
                                            class_mode="binary",
                                            target_size=(224,224),
                                            color_mode='rgb')

Found 1235 validated image filenames belonging to 2 classes.
Found 411 validated image filenames belonging to 2 classes.


In [64]:
# test data generator
test_datagen = ImageDataGenerator(rescale=1./255.,
                             rotation_range=30,
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                             shear_range=0.2,
                             zoom_range=0.2,
                             channel_shift_range=0.2,
                             horizontal_flip=True,
                             vertical_flip=True)
test_generator = test_datagen.flow_from_dataframe(dataframe=test_img,
                                                  directory=img_dir,
                                                  x_col="image_names",
                                                  y_col=None,
                                                  batch_size=32,
                                                  seed=42,
                                                  shuffle=False,
                                                  class_mode=None,
                                                  target_size=(224,224),
                                                  color_mode='rgb')

Found 706 validated image filenames.


## CNN model

In [0]:
# # initiate sequential model
# model = Sequential()
# # add convolutional layer
# # 16 sliding windows each of 3X3 size
# # default step is 1X1
# model.add(Conv2D(filters = 32,
#                  kernel_size = (5, 5),
#                  activation='relu',
#                  input_shape = (224, 224,3),
#                  padding='same'))
# # add batch normalization to normalize output of the layer
# model.add(BatchNormalization())
# # add another convolutional layer
# model.add(Conv2D(filters = 32,
#                  kernel_size = (5, 5),
#                  activation='relu',
#                  padding='same'))
# # batchnormalize
# model.add(BatchNormalization())
# # add maxpooling layer
# # this layer picks max value for every 2X2 window
# model.add(MaxPool2D(pool_size=(2,2)))
# # add dropout layer
# model.add(Dropout(0.3))
# # repeat above sequence once more
# model.add(Conv2D(filters = 64,
#                  kernel_size = (5, 5),
#                  activation='relu',
#                  padding='same'))
# model.add(BatchNormalization())
# model.add(Conv2D(filters = 64,
#                  kernel_size = (5, 5),
#                  activation='relu',
#                  padding='same'))
# model.add(BatchNormalization())
# model.add(MaxPool2D(pool_size=(2,2)))
# model.add(Dropout(0.3))
# # another pair of convolutional layers
# model.add(Conv2D(filters = 128,
#                  kernel_size = (5, 5),
#                  activation='relu',
#                  padding='same'))
# model.add(BatchNormalization())
# model.add(Conv2D(filters = 128,
#                  kernel_size = (5, 5),
#                  activation='relu',
#                  padding='same'))
# model.add(BatchNormalization())
# model.add(Conv2D(filters = 128,
#                  kernel_size = (5, 5),
#                  activation='relu',
#                  padding='same'))
# model.add(BatchNormalization())
# model.add(MaxPool2D(pool_size=(2,2)))
# model.add(Dropout(0.3))
# # flatten cnn layers
# model.add(Flatten())
# # add dense layer
# model.add(Dense(512, activation='relu'))
# model.add(Dropout(0.3))
# model.add(Dense(256, activation='relu'))
# model.add(Dropout(0.3))
# # finally add a softmax layer which will predict probability of each class
# model.add(Dense(1, activation='sigmoid'))
# # print model summary
# model.summary()

# # compile model
# model.compile(loss='binary_crossentropy',
#              optimizer='adam',
#              metrics=['accuracy'])

## InceptionV3

In [66]:
# Setup transfer learning model

# # load model without classifier layers
# base_model = VGG16(include_top=False,
#                    input_tensor=Input(shape=(224, 224, 3)),
#                    weights='imagenet')
# # add new classifier layers
# flat1 = Flatten()(base_model.outputs)
# #flat1 = GlobalAveragePooling2D()(flat1)
# class1 = Dense(256, activation='relu')(flat1)
# output = Dense(1, activation='sigmoid')(class1)
# # define new model
# model = Model(inputs=base_model.inputs, outputs=output)
# # summarize
# model.summary()

# # first: train only the top layers (which were randomly initialized)
# # i.e. freeze all convolutional ResNet50 layers
# for layer in base_model.layers:
#     layer.trainable = False

model = Sequential()
# NOTE that this layer will be set below as NOT TRAINABLE, i.e., use it as is
model.add(InceptionV3(include_top = False, input_shape = (224,224,3)))
# model.add(GlobalAveragePooling2D())
model.add(Flatten())
# 2nd layer as Dense for 2-class classification, i.e., dog or cat using SoftMax activation
model.add(Dense(1024, activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation = 'sigmoid'))

# Say not to train first layer (ResNet) model as it is already trained
# model.layers[0].trainable = False
model.summary()
# optimizer
opt = adam(learning_rate=0.0001)
# compile model
model.compile(loss='binary_crossentropy',
             optimizer=opt,
             metrics=['accuracy'])

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 5, 5, 2048)        21802784  
_________________________________________________________________
flatten_5 (Flatten)          (None, 51200)             0         
_________________________________________________________________
dense_13 (Dense)             (None, 1024)              52429824  
_________________________________________________________________
dropout_9 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_14 (Dense)             (None, 512)               524800    
_________________________________________________________________
dropout_10 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 1)                

## Train CNN model

In [0]:
STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size
STEP_SIZE_VALID = valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

In [68]:
model.fit_generator(generator=train_generator,
                    validation_data=valid_generator,
                    epochs=20)

Epoch 1/20
39/39 [==============================] - 49s 1s/step - loss: 0.8772 - accuracy: 0.6923 - val_loss: 0.3390 - val_accuracy: 0.8686
Epoch 2/20
39/39 [==============================] - 23s 598ms/step - loss: 0.3774 - accuracy: 0.8526 - val_loss: 0.3463 - val_accuracy: 0.8662
Epoch 3/20
39/39 [==============================] - 25s 644ms/step - loss: 0.3029 - accuracy: 0.8866 - val_loss: 0.2148 - val_accuracy: 0.9197
Epoch 4/20
39/39 [==============================] - 25s 639ms/step - loss: 0.2635 - accuracy: 0.8980 - val_loss: 1.3013 - val_accuracy: 0.9270
Epoch 5/20
39/39 [==============================] - 25s 636ms/step - loss: 0.2016 - accuracy: 0.9223 - val_loss: 0.1759 - val_accuracy: 0.8978
Epoch 6/20
39/39 [==============================] - 25s 640ms/step - loss: 0.1562 - accuracy: 0.9344 - val_loss: 0.0420 - val_accuracy: 0.9270
Epoch 7/20
39/39 [==============================] - 25s 633ms/step - loss: 0.1239 - accuracy: 0.9603 - val_loss: 0.4397 - val_accuracy: 0.9270
Ep

In [69]:
model.evaluate_generator(generator=valid_generator,
                         steps=STEP_SIZE_TEST)

[0.8189684152603149, 0.9399141669273376]

## Submissions

In [70]:
# image genrator
datagen_final = ImageDataGenerator(rescale=1./255.,
                             rotation_range=30,
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                             shear_range=0.2,
                             zoom_range=0.2,
                             channel_shift_range=0.2,
                             horizontal_flip=True,
                             vertical_flip=True)
# train generator
train_generator_final = datagen_final.flow_from_dataframe(dataframe=train_img,
                                            directory=img_dir,
                                            x_col="image_names",
                                            y_col="emergency_or_not",
                                            batch_size=32,
                                            seed=42,
                                            shuffle=True,
                                            class_mode="binary",
                                            target_size=(224,224),
                                            color_mode='rgb')
# final model
final_model = Sequential()
# NOTE that this layer will be set below as NOT TRAINABLE, i.e., use it as is
final_model.add(InceptionV3(include_top = False, input_shape = (224,224,3)))
# model.add(GlobalAveragePooling2D())
final_model.add(Flatten())
# 2nd layer as Dense for 2-class classification, i.e., dog or cat using SoftMax activation
final_model.add(Dense(1024, activation = 'relu'))
final_model.add(Dropout(0.3))
final_model.add(Dense(512, activation = 'relu'))
final_model.add(Dropout(0.3))
final_model.add(Dense(1, activation = 'sigmoid'))

# Say not to train first layer (ResNet) model as it is already trained
# model.layers[0].trainable = False
final_model.summary()
# optimizer
opt = adam(learning_rate=0.0001)
# compile model
final_model.compile(loss='binary_crossentropy',
             optimizer=opt,
             metrics=['accuracy'])

Found 1646 validated image filenames belonging to 2 classes.
Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 5, 5, 2048)        21802784  
_________________________________________________________________
flatten_6 (Flatten)          (None, 51200)             0         
_________________________________________________________________
dense_16 (Dense)             (None, 1024)              52429824  
_________________________________________________________________
dropout_11 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_17 (Dense)             (None, 512)               524800    
_________________________________________________________________
dropout_12 (Dropout)         (None, 512)               0         
___________________________________________________________

In [71]:
final_model.fit_generator(generator=train_generator_final, epochs=20)

Epoch 1/20
52/52 [==============================] - 46s 884ms/step - loss: 0.6771 - accuracy: 0.7199
Epoch 2/20
52/52 [==============================] - 28s 545ms/step - loss: 0.3033 - accuracy: 0.8797
Epoch 3/20
52/52 [==============================] - 28s 530ms/step - loss: 0.2225 - accuracy: 0.9192
Epoch 4/20
52/52 [==============================] - 28s 540ms/step - loss: 0.1896 - accuracy: 0.9289
Epoch 5/20
52/52 [==============================] - 28s 532ms/step - loss: 0.1553 - accuracy: 0.9362
Epoch 6/20
52/52 [==============================] - 28s 536ms/step - loss: 0.1476 - accuracy: 0.9490
Epoch 7/20
52/52 [==============================] - 27s 528ms/step - loss: 0.1388 - accuracy: 0.9441
Epoch 8/20
52/52 [==============================] - 28s 537ms/step - loss: 0.1070 - accuracy: 0.9581
Epoch 9/20
52/52 [==============================] - 28s 531ms/step - loss: 0.1014 - accuracy: 0.9642
Epoch 10/20
52/52 [==============================] - 27s 528ms/step - loss: 0.0869 - accura

In [72]:
test_generator.reset()
pred = final_model.predict_generator(test_generator,
                             verbose=1)

23/23 [==============================] - 11s 469ms/step


In [73]:
pred.shape

(706, 1)

In [74]:
pred[pred >= 0.5] = 1
pred[pred < 0.5] = 0
pred[:10]

array([[0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.]], dtype=float32)

In [0]:
pred = pred.ravel()

In [0]:
# labels = (train_generator.class_indices)
# labels = dict((v,k) for k,v in labels.items())
# predictions = [labels[k] for k in predicted_class_indices]

In [0]:
out_dir = '/content/drive/My Drive/JantaHack Computer Vision/'
filenames=test_generator.filenames
results=pd.DataFrame({"image_names":filenames,
                      "emergency_or_not":pred})
results.to_csv(out_dir+"Submissions5.csv",index=False)